## Artificial Neural Network

This projects deals with building an Artificial Neural Network and understanding why Neural Networks outperform most of the other simpler Machine Learning Algorithms.

The goal of ANN is to build a network of neurons with 1 or more hidden layers of neurons to allow for backpropagation of synapses that improves the performance/accuracy of the model. The goal of this type of models is to constantly keep checking for loss function which is the difference between predicted y_hat and actual Y to adjust the weights at the input nodes and hidden nodes.

Lets start with importing the necessary modules.

First we need to install the KERAS library, Tensorflow and Theano done using the below commands

In [1]:
!pip install --upgrade keras
!pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git
!pip install tensorflow

Requirement already up-to-date: keras in c:\users\rames\anaconda3\lib\site-packages
Requirement already up-to-date: pyyaml in c:\users\rames\anaconda3\lib\site-packages (from keras)
Requirement already up-to-date: scipy>=0.14 in c:\users\rames\anaconda3\lib\site-packages (from keras)
Requirement already up-to-date: six>=1.9.0 in c:\users\rames\anaconda3\lib\site-packages (from keras)
Requirement already up-to-date: numpy>=1.9.1 in c:\users\rames\anaconda3\lib\site-packages (from keras)
  Cloning git://github.com/Theano/Theano.git to c:\users\rames\appdata\local\temp\pip-11cdfxm6-build


  Error [WinError 2] The system cannot find the file specified while executing command git clone -q git://github.com/Theano/Theano.git C:\Users\rames\AppData\Local\Temp\pip-11cdfxm6-build
Cannot find command 'git'


## Importing the data and performing some preprocessing to clean the data

The data that we would be using is a sample of a fictional dataset. The data fictiously represents a bank data with geo demographical information of the customers such as 
1. customerId, 
2. Last Name, 
3. CreditScore, 
4. Geography( Here France, Spain and Germany), 
5. Gender, 
6. Age, 
7. Tenure, 
8. Current Balance in accounts, 
9. Number of Products the customer currently has, 
10. Whether he has credit card with the bank, 
11. Is he an active customer (last 2 months), 
12. Estimated Salary

Finally the dependent variable 
Exited - 1 denoting he stopped being a customer and 0 representing still a customer

A copy of the csv is available in here:(Add link here)

In [2]:
# Lets import the data
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


Lets split the data into input nodes and output nodes.
X represent the input nodes. 
Y represent the output nodes.


In [3]:
X = dataset.iloc[:, 3:13].values
Y = dataset.iloc[:, 13].values

### Dealing with Categorical data

Since out input data has two categorical data in Geography and Gender, we need to convert them into numerical fields. We can use One Hot Encoder to do that and then remove one of the Geography encoded values to avoid Dummy Variable Trap

In [4]:
# Encoding categorical data
# Encoding the Independent Variable
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()

X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])

labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])

onehotencoder = OneHotEncoder(categorical_features = [1])

X = onehotencoder.fit_transform(X).toarray()

X=X[:, 1:]

### Split the available data into Test and Train

We can directly use train test split from the sklearn to split 20% of the data as test data for our evaluations later on.

In [5]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=0)

### Feature Scaling

Since our Output node is a binary outcome, we will be using the sigmoid/logistic activation function in our output node. Therefore it is essential for us to standardize the independent variables to allow for the sigmoid function to capture the variance better. Here we are going to do that using StandardScalar.


In [6]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.fit_transform(X_test)

In [7]:
Y_train.shape

(8000,)

In [8]:
X_train.shape

(8000, 11)

## Building an Artificial Neural Network

Now we can start building an artificial neural Network

Step one is to import all the necessary modules from the Keras library. We are importing Sequential and Dense 

Step 2 is to initiate a Neural Network which is done using Sequential function. This signifies that our neural nodes would be sequentially arranged

Step 3 is to initiate our first hidden layer
* We use the add function to add our first hidden layer

* Here we need to specify the number of output dimensions, which would be the number of nodes in our first hidden layer. As a general rule of thumb, it is acceptable to use the average of input and output nodes as "the" number of nodes here. Since we have 1 output node and 11 input nodes, we use an average of those two numbers to determine the number of nodes in our first hidden layer

* The next thing we need to specify is the initialization function . Uniform allows us to associate a small non zero value to the initial weights for the input.

* I am adding a rectifier function as activation function for our first hidden layer

* Finally the input dimentions in our case =11

Step 4 is to initialize the second hidden layer
* Using the same number of nodes as the first layer
* Using the same rectifier activation function


Step 5 - Define the output Node
* Our Output node has 1 node since this is a binary output of 1 or 0
* Our activation function, of course, is a sigmoid function since it gives a nice probability that can be used to convert to 0 or 1 using a threshold

In [9]:
# Importing the keras libraries to create out Neural Networks

import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [10]:
# Defining the initial object as a sequential model with more than one layers
clf=Sequential()

# Adding the first hidden layer
# output_dim is the average of input dims=11 and output dims=1.
# init is the function to imitialize the weights. uniform is the most simple way to add a close to 0 value for weights 
# Choosing the activation function to be relu which corresponds to rectifier function 
clf.add(Dense(units=6,kernel_initializer="uniform",activation='relu',input_dim=11))

#Adding the second hidden layer
clf.add(Dense(units=6,kernel_initializer="uniform",activation='relu'))

# Adding the final output layer
clf.add(Dense(units=1,kernel_initializer="uniform",activation='sigmoid'))

## Compile the whole ANN.. 

* Applying Stochastic Neural Network as our optimizer which is the optimizing algorithm that is used for weight calculation
* So far the weights have only been initialized, so the Stochastic algorithm would be used for weight estimation/calculation . Done here using ADAM
* Loss represents the loss function that will used to find the optimized solution
* Metrics are the criterion that are used to evaluate the model

In [11]:
clf.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [12]:
# Prediction and model evaluation

clf.fit(X_train,Y_train,batch_size=10,epochs=100)


Epoch 1/100
8000/8000 [==============================] - 0s - loss: 0.4819 - acc: 0.7959     
Epoch 2/100
8000/8000 [==============================] - 0s - loss: 0.4286 - acc: 0.7960     
Epoch 3/100
8000/8000 [==============================] - 0s - loss: 0.4231 - acc: 0.7960     
Epoch 4/100
8000/8000 [==============================] - 0s - loss: 0.4191 - acc: 0.8199     
Epoch 5/100
8000/8000 [==============================] - 0s - loss: 0.4162 - acc: 0.8292     
Epoch 6/100
8000/8000 [==============================] - 0s - loss: 0.4143 - acc: 0.8304     
Epoch 7/100
8000/8000 [==============================] - 0s - loss: 0.4129 - acc: 0.8336     
Epoch 8/100
8000/8000 [==============================] - 0s - loss: 0.4118 - acc: 0.8320     
Epoch 9/100
8000/8000 [==============================] - 0s - loss: 0.4108 - acc: 0.8336     
Epoch 10/100
8000/8000 [==============================] - 0s - loss: 0.4096 - acc: 0.8339     
Epoch 11/100
8000/8000 [==============================] - 0

8000/8000 [==============================] - 0s - loss: 0.4003 - acc: 0.8335     
Epoch 87/100
8000/8000 [==============================] - 0s - loss: 0.4002 - acc: 0.8344     
Epoch 88/100
8000/8000 [==============================] - 0s - loss: 0.3999 - acc: 0.8355     
Epoch 89/100
8000/8000 [==============================] - 0s - loss: 0.4008 - acc: 0.8339     
Epoch 90/100
8000/8000 [==============================] - 0s - loss: 0.4004 - acc: 0.8346     
Epoch 91/100
8000/8000 [==============================] - 0s - loss: 0.4006 - acc: 0.8352     
Epoch 92/100
8000/8000 [==============================] - 0s - loss: 0.4001 - acc: 0.8351     
Epoch 93/100
8000/8000 [==============================] - 0s - loss: 0.3998 - acc: 0.8347     
Epoch 94/100
8000/8000 [==============================] - 0s - loss: 0.4004 - acc: 0.8356     
Epoch 95/100
8000/8000 [==============================] - 0s - loss: 0.4000 - acc: 0.8354     
Epoch 96/100
8000/8000 [==============================] - 0s - 

In [13]:
# Applying the model on the split test data
Y_pred=clf.predict(X_test)
Y_pred=(Y_pred>0.5)

# Calculating the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(Y_test,Y_pred)
cm

array([[1548,   47],
       [ 272,  133]], dtype=int64)

Looking at the above results we can infer that the model consistently predicts with an accuracy of about 83% in the train dataset. From the confusion Matrix we can calcualte the accuracy in the test data as (1548+133)/2000 = 84%.

Based on the above the model is pretty robust with consistent accuracy metrics leading to believe that there is no over fitting